In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy
from torch.nn import functional as F

In [3]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in charactersL ", len(text))


length of dataset in charactersL  1115394


In [6]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [31]:
#Create a mapping (encoder and decoder)
stoi = {ch:i for i,ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:  [stoi[c] for c in s]
decode = lambda l: ''.join([itos[c] for c in l])

In [32]:
#Tokenize all data
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [33]:
n= int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [34]:
block_size = 8 #Contextlength
train_data[:block_size+1] #In 9 characters there are 8 examples because 18->47, 47->56 ...

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [35]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When the input is {context} the target is: {target}")

When the input is tensor([18]) the target is: 47
When the input is tensor([18, 47]) the target is: 56
When the input is tensor([18, 47, 56]) the target is: 57
When the input is tensor([18, 47, 56, 57]) the target is: 58
When the input is tensor([18, 47, 56, 57, 58]) the target is: 1
When the input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
When the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
When the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [36]:
torch.manual_seed(1337)
batch_size = 4  #How many indepedent sequences will be processed in parallel??
block_size = 8  #What is the maximum context length for hte predictions??
                #We can have 1 character of context. Can predict up to block_Size
                #No more than block size inputs??

def get_batch(split):
    #Generate a small batch of data of inputs x and targets y
    data =  train_data if split == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #ix is 4 numbers that will be randmoly generated between 0 and len(data)-blocksize
    #Random off sets
    x = torch.stack([data[i:i+block_size] for i in ix]) #stack them as rows
    y = torch.stack([data[i+1: i+block_size+1] for i in ix]) #stack them as rows
    #Will become a 4 x 8 tensor
    return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('----')
for b in range(batch_size): #batch dimension
    for t in range(block_size): #time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"When input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
When input is [24] the target: 43
When input is [24, 43] the target: 58
When input is [24, 43, 58] the target: 5
When input is [24, 43, 58, 5] the target: 57
When input is [24, 43, 58, 5, 57] the target: 1
When input is [24, 43, 58, 5, 57, 1] the target: 46
When input is [24, 43, 58, 5, 57, 1, 46] the target: 43
When input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
When input is [44] the target: 53
When input is [44, 53] the target: 56
When input is [44, 53, 56] the target: 1
When input is [44, 53, 56, 1] the target: 58
When input is [44, 53, 56, 1, 58] the target: 46
When input is [44, 53

In [37]:
#Bigram model
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) #num_embedding, embdedding_dim
        #chooses a row corresponding to index

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) # B,T,C #We want to arrange this into b,t,c
        #Batch=4, Time=8(block), Channels=vocab_size=65
        B,T,C= logits.shape

        if targets is None:
            loss=None
        else:
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):

        #idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get predictions : )))
            logits, loss = self(idx)
            #FOCUS ONLY ON THE LASST TIME STEP

            logits = logits[:, -1, :] #becomes B,C
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) #B,C
            idx_next = torch.multinomial(probs, num_samples=1) #B,1
            idx = torch.cat((idx, idx_next), dim=1) #B, T+1
        return idx
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()) )

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [38]:
optim = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [47]:
batch_size=32
for steps in range(10000):
    xb,yb = get_batch('train')

    logits, loss = m(xb,yb)
    optim.zero_grad(set_to_none=True)
    loss.backward()
    optim.step()
print(loss.item())

2.4966657161712646


In [51]:
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()) )



BAn d te, ucout Ye anthivieat it gher
ARKEDYobengou wan?
Yeit; of y te toee mpirealis, se LELESe bue


In [52]:
#We need to get the tokens to start talking to each other!
#MATH TRICK
torch.seed(1)
B,T,C = 4,8,2 #GET THE Ts to talk, but not talk to future tokens...
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [59]:
#Let's do average first of previous tokens.
xbow = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # t , c
        xbow[b,t] = torch.mean(xprev, 0)

In [60]:
#Efficiency with matrix multiplication
a=torch.tril(torch.ones(3,3)) ####THE TRIL ACTS A BOOL OF WHO TO IGNORE
a=a/torch.sum(a, dim=1, keepdim=True)
b = torch.randint(0,10,(3,2)).float() #EACH COLUMN OF B IS BATCH??????????
c=a@b
print(a)
print(b)
print(c)

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
tensor([[4., 4.],
        [8., 8.],
        [0., 8.]])
tensor([[4.0000, 4.0000],
        [6.0000, 6.0000],
        [4.0000, 6.6667]])


In [61]:
#Now one our B,T,C
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1,keepdim=True)
xbow2 = wei @ x #(T,T) @ (B,T,C) --> pytorch will make batch dimesnion
# (B,T,T) @ (B,T,C) -> (B,T,C) easy!
torch.allclose(xbow,xbow2)

True

In [62]:
#VERSION 3
tril=torch.tril(torch.ones(T,T))
wei=torch.zeros(T,T) #how much of each of the tokens should we look at ...?
wei=wei.masked_fill(tril==0, float('-inf'))
wei=F.softmax(wei,dim=1)
xbow3= wei@x
torch.allclose(xbow,xbow3)

True

In [69]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [72]:
#VERSION 4
torch.manual_seed(1337)
B,T,C=4,8,32
x=torch.randn(B,T,C)
#IMPLEMENT HEAD
head_size=16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) #B,T,head_size
q = query(x) #B, T, head_size
v = value(x)
wei = q @ k.transpose(-2,-1) #B,T,H @ B,H,T --> B, T, T

#need to preserve variance so needs to be scaled dot product
#---
tril=torch.tril(torch.ones(T,T))
#wei=torch.zeros(T,T) #how much of each of the tokens should we look at ...?
#WE DONT WANT WEI TO BE UNIFORM! #want to let the tokens choose interesting tokens
#want to gather data from the past but in a data dependent way
wei=wei.masked_fill(tril==0, float('-inf'))
wei=F.softmax(wei,dim=-1)

out = wei @ v
out.shape ###B,T,H=V=heads=16

#Every single token will emit 2 vectors, query and a key.
#query - what am i looking for
#key - what do i contain
#to get affinities, dot product of keys and queries.
#THAT DOT PRODUCT BECOMES WEI

torch.Size([4, 8, 16])

In [71]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4040, 0.5960, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0231, 0.5293, 0.4476, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2742, 0.4263, 0.1339, 0.1655, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2915, 0.0823, 0.1611, 0.2318, 0.2332, 0.0000, 0.0000, 0.0000],
        [0.0238, 0.0941, 0.1523, 0.5596, 0.0103, 0.1599, 0.0000, 0.0000],
        [0.0051, 0.2255, 0.0492, 0.6207, 0.0024, 0.0643, 0.0327, 0.0000],
        [0.0067, 0.0415, 0.0337, 0.6519, 0.0058, 0.1384, 0.0519, 0.0700]],
       grad_fn=<SelectBackward0>)

In [73]:
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [75]:
B,T,C = 4,8,8 #GET THE Ts to talk, but not talk to future tokens...
x = torch.randn(B,T,C)
k = k.view(B, T, 4, C // 4).transpose(1, 2) # (B, nh, T, hs)


RuntimeError: shape '[4, 8, 4, 2]' is invalid for input of size 512